# 多动作多智能体-软件公司

目标：实现一个软件公司，它可以对外投资，员工状态，项目进度等。

**安装依赖**

1.安装metagpt
```python
pip install metagpt
```
2.安装 pydantic（如果尚未安装）
```python
pip install pydantic==2.9.4 #pydantic级别高版本会报错
```

**插入模块**

In [ ]:
from pydantic import BaseModel, Field
from metagpt.actions import BossRequirement
from metagpt.config import CONFIG
from metagpt.environment import Environment
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.utils.common import NoMoneyException

In [ ]:
**定义角色**

1.创建软件公司基础模型

```python
class SoftwareCompany(BaseModel):
    """
    Software Company: Possesses a team, SOP (Standard Operating Procedures), and a platform for instant messaging,
    dedicated to writing executable code.
    """
    environment: Environment = Field(default_factory=Environment)
    investment: float = Field(default=10.0)
    idea: str = Field(default="")

    class Config:
        arbitrary_types_allowed = True
```

**定义动作**

在创建的软件公司模型中我们定义了几个动作，包括：雇用员工，对外投资，检查预算平衡，发起项目。

```python
#雇佣员工
def hire(self, roles: list[Role]):
    """Hire roles to cooperate"""
    self.environment.add_roles(roles)
#对外投资
def invest(self, investment: float):
    """Invest company. raise NoMoneyException when exceed max_budget."""
     self.investment = investment
     CONFIG.max_budget = investment
     logger.info(f'Investment: ${investment}.')
#检查预算平衡
def _check_balance(self):
    if CONFIG.total_cost > CONFIG.max_budget:
        raise NoMoneyException(CONFIG.total_cost, f'Insufficient funds: {CONFIG.max_budget}')
#启动项目
def start_project(self, idea):
    """Start a project from publishing boss requirement."""
    self.idea = idea
    self.environment.publish_message(Message(role="BOSS", content=idea, cause_by=BossRequirement))
#保存公司状态
def _save(self):
    logger.info(self.json())
```

定义软件公司运作，直至目标周转或者公司破产

```python
# 软件公司运作
async def run(self, n_round=3):
    """Run company until target round or no money"""
    while n_round > 0:
        # self._save()
        n_round -= 1
        logger.debug(f"{n_round=}")
        self._check_balance()
        await self.environment.run()
    return self.environment.history
```
# 定义员工角色
```python
# 定义员工角色
class Engineer(Role):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([BossRequirement])

    async def _act(self) -> Message:
        logger.info(f"{self.name} is working on the project.")
        return Message(content="Code implemented successfully.", role=self.name)
```
**主函数**
```python
async def main():
    # 初始化软件公司
    company = SoftwareCompany()

    # 投资公司
    company.invest(investment=20.0)

    # 雇用员工
    engineer1 = Engineer(name="Engineer1")
    engineer2 = Engineer(name="Engineer2")
    company.hire(roles=[engineer1, engineer2])

    # 启动项目
    company.start_project(idea="Build a new web application.")

    # 运行公司
    history = await company.run(n_round=5)
    logger.info("Company run completed.")
    logger.info(f"History: {history}")


if __name__ == "__main__":
    # 使用 asyncio 运行主函数
    asyncio.run(main())
```
**运行代码**
在终端中运行以下命令启动软件公司 ：
```python
python software_company.py
```
**运行结果**
运行后，终端会打印出以下信息：

1.公司投资金额。

2.员工的工作状态（例如：Engineer1 is working on the project.）。

3.项目完成后的历史记录